In [1]:
with open("mod_hyde/data/all_book_data.txt","r") as f:
    all_books_data = f.read() 

In [2]:
import json

youtube_transcripts_list = ["chunked_misc_transcripts.json","chunked_transcripts_undergrad.json","chunked_transcripts_mba.json"]    
all_youtube_data = []
for file in youtube_transcripts_list:
    with open('mod_hyde/data/YouTube_API_Transcripts/'+file) as f:
        data = json.load(f)
    for id,data in data.items():
        for d in data:
            all_youtube_data.append(d['text'])

In [3]:
import re

all_books_data_list = re.split(r"(?<=[.?!])\s+", all_books_data)

## CHUNKING 

In [2]:
!pip install -Uqq langchain

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    separators = [". "],
    chunk_size=1750,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,

)

books_data_splitted_text = text_splitter.split_text(all_books_data)

In [5]:
all_data = books_data_splitted_text + all_youtube_data

## DISTILBERT

In [7]:
from transformers import AutoTokenizer

import pandas as pd

df = pd.DataFrame(all_data,columns=["text"])

def tokenize_function(examples):
    return tokenizer(examples["text"])

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
from datasets import Dataset
import pandas as pd

hf_dataset = Dataset.from_pandas(df)

tokenized_datasets = hf_dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

/home/athekunal/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Map (num_proc=4): 100%|██████████| 8394/8394 [00:03<00:00, 2659.38 examples/s]


In [8]:
block_size = 1024
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)


Map (num_proc=4): 100%|██████████| 8394/8394 [00:10<00:00, 808.40 examples/s] 


In [18]:
# lm_datasets[0]['input_ids']

In [17]:
lm_datasets[0]['labels'] == lm_datasets[0]['input_ids']

True

In [6]:
# from transformers import AutoTokenizer, AutoModelForCausalLM

# model_name = "distilbert/distilgpt2"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)

In [21]:
all_books_data_list[1]

'These stories not only sound persuasive and reasonable but are also backed up by evidence - anecdotal, in some cases, and statistical, in others - that the strategies work.'